<img src="./img/hpe_logo.png" alt="HPE Logo" width="300">

<h1>Demo of the Enterprise Machine Learning/Data Science Platform</h1>

<h5>Date: 08/24/23</h5>
<h5>Version: 1.0</h5>
<h5>Author(s): andrew.mendez

# Overview
## We will build an End to End ML Pipeline to train and deploy AI to detect Aircraft in Satellite Imagery
<img src="./img/model-pred.png" alt="Enterprise Machine Learning platform architecture" width="850">


# How will we build this?
<img src="./img/PDK_Demo_Overview.png" alt="Enterprise Machine Learning platform architecture" width="850">

# What are they Key Challenges with delivering world class AI at the speed of the mission

## Going from a research project to production
## How can you equip teams of researchers

* Good Data, Good Models, and Good Infrastructure
* **Good Infrastructure is the hardest, and the biggest issue that cause 80-90% of ML projects to fail**


What does Good AI Infrastructure Look Like:
* Data Infrastructure:
    * Data Management
    * Data Versioning
    * Pipeline Orchestration
* Training Infrastructure
    * Resource Management
    * Distributed Training
    * Fault Tolerance and Resumption
    * Experiment Tracking
    * IDE for developers to develop models
    * Logging and Visualizing Metrics
    * Hyper-parameter search
* Serving Infrastructure
    * Automatic Deployments
    * Versioning Models
    * Automatic Resource Allocation



# How the HPE Machine Learning/Data Science Platform Helps:

Built off two leading AI software Pachyderm and Determined.AI via acquisition.

## What does the HPE ML/DS Plaform provide:
* Data Infrastructure (Pachyderm):
    * Data Management
    * Data Versioning
    * Pipeline Orchestration
* Training Infrastructure (Determined.AI)
    * Resource Management
    * Distributed Training
    * Multi-node training (LLMs)
    * Fault Tolerance and Resumption at scale
    * Experiment Tracking
    * IDE for developers to develop models
    * Logging and Visualizing Metrics
    * Hyper-parameter search at scale
* Serving Infrastructure (KServe)
    * Automatic Deployments
    * Versioning Models
    * Automatic Resource Allocation
    
## End Result:
* Less Lines of Code to manage data, train, and deploy at scale
* Out of the box support for distributed training, hyperparameter search, and experiment tracking
* Ensure the productivity and success of your team delivering great models at the speed of the mission

<img src="./img/platform_step3.png" alt="Enterprise Machine Learning platform architecture" width="850">

# Outline:
* Create Data Repo and Upload data
* Data Exploration
* Create Simple Baseline
* Train Model on MLDE
* Migrate Training and Inference into Scalable, Automated E2E Ops: Model deployment for inference
* Trigger End to End Pipeline Execution with updated better data
* Trigger End to End Pipeline Execution with updated dataset

<h3>Import modules and define functions</h3>
The cell below imports all modules and libraries required to run the demo.

In [ ]:
# General modules
import os
import random
import numpy as np
import time
# Torch modules
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision import models, transforms
import json
# Image modules
from PIL import Image
from skimage import io
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Import functions for downloading data
from load_data import download_pach_repo, download_data, get_train_transforms

# Import Determined Client
from determined.experimental import client as det
from determined import pytorch
from utils import calculate_coco_stats, visualize_coco_annotations, load_model
# Remove warnings
import warnings

import python_pachyderm
from python_pachyderm.service import pps_proto

warnings.filterwarnings('ignore')

<h1>Interactive Experimentation </h1>

<img src="./img/platform_step1.png" alt="Enterprise Machine Learning platform architecture" width="850">

In [ ]:
# Define variables for Pachyderm connection
pachyderm_host = "10.182.1.45"
pachyderm_port = 80
repo = "data"
data_dir = './data/xview_3class_full/'
branch = "master"
project = "object-detection-demo"
download_dir = "./data_downloaded"
token = " "

In [ ]:
# create new project
client = python_pachyderm.Client(
    host=pachyderm_host, port=pachyderm_port, auth_token=token
)
# client.create_project(project)# Un-comment if you have not already created a project named: object-detection-demo


# Create Data Repo and Upload data to Pachyderm

In [ ]:
try:
    client.create_repo(repo,project_name=project)# This line creates a Pachyderm Repo
except Exception as e:
    print("Failed to create data repo: {}...will try to upload to existing repo".format(repo))
    pass

source_dir  = data_dir
# Populate the input repos
def insert_data(client,name,source_data_dir,project):
    print("Inserting {} data...".format(name))
    with client.commit(name, "master",project_name=project) as c:
        # data_dir = "{}_data".format(name)
        python_pachyderm.put_files(client, source_data_dir, c, "/")
                
        return c

d_commit = insert_data(client,repo,source_data_dir=source_dir,project=project)

# Wait for the commits to finish
print("Waiting for commits to finish...")
for commit in [client.wait_commit(c.id)[0] for c in [d_commit]]:
    print(commit)

<h3>Inspect first version of data</h3>

In [ ]:
# Call download, store paths in files
files = download_data(pachyderm_host, pachyderm_port, repo, branch, project, download_dir, token)

<h3> Some light data exploration</h3>

In [ ]:
FILE_PATH='./data_downloaded/train_images_rgb_no_neg_filt_32/train_640_02_filtered_32.json'
(num_annotations, 
 cat_ids, 
 cat_names, 
 annotations_per_category, 
 min_annotations, 
 max_annotations, 
 average_annotations, 
 min_annotation_area,
 max_annotation_area,
 avg_annotation_area,
 min_annotation_area_per_category, 
 max_annotation_area_per_category) = calculate_coco_stats(FILE_PATH)
print("Number of images:", num_annotations)
print(f"Number of Unique Categories: {len(cat_ids)}")
print(cat_ids)  # The IDs are not necessarily consecutive.

print("Category IDs:")
print("Categories Names: ", cat_names)
# Print or use annotations_per_category as needed
print("Number of annotations per category:")
print(annotations_per_category)
# Print or use the calculated metrics as needed
print("Minimum number of annotations for an image:", min_annotations)
print("Maximum number of annotations in an image:", max_annotations)
print("Average number of annotations per image:", average_annotations)

print("Minimum annotation area:", min_annotation_area)
print("Maximum annotation area:", max_annotation_area)
print("Average annotation area:", avg_annotation_area)

print("\nMinimum annotation area per category:")
print(min_annotation_area_per_category)

print("\nMaximum annotation area per category:")
print(max_annotation_area_per_category)

In [ ]:
coco_ann_file_path = './data_downloaded/train_images_rgb_no_neg_filt_32/train_640_02_filtered_32.json'
coco_img_dir = './data_downloaded/train_images_rgb_no_neg_filt_32/train_images_640_02_filt_32'
visualize_coco_annotations(coco_ann_file_path, coco_img_dir)

In [ ]:
import datetime
import os
import time

import torch
import torch.utils.data
import torchvision
import torchvision.models.detection
import torchvision.models.detection.mask_rcnn

# from detection_utils.coco import get_coco, get_coco_kp
from detection_utils.data import build_xview_dataset_filtered
from detection_utils.group_by_aspect_ratio import GroupedBatchSampler, create_aspect_ratio_groups
from detection_utils.engine import train_and_eval, eval_model

# from detection_utils.train import get_dataset, get_transform
from detection_utils.models import build_frcnn_model
from PIL import Image
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from collections import OrderedDict
from tqdm import tqdm
import numpy as np
from attrdict import AttrDict
import matplotlib.pyplot as plt

<h2> Create Simple Baseline</h2>

### Create data set class (inherit from Pytorch dataset)
```python
class LocalBackend:
    """
    This class will load data from harddrive.
    COCO dataset will be downloaded from source in model_def.py if
    local backend is specified.
    """

    def __init__(self, outdir):
        assert os.path.isdir(outdir)
        self.outdir = outdir

    def get(self, filepath):
        with open(os.path.join(self.outdir, filepath), "rb") as f:
            img_str = f.read()
        return img_str


class CocoDetection(torchvision.datasets.CocoDetection):
    def __init__(
        self,
        backend,
        root_dir,
        img_folder,
        ann_file,
        transforms,
        return_masks,
        catIds=[],
    ):
        super(CocoDetection, self).__init__(img_folder, ann_file)
        self.img_folder = img_folder
        self._transforms = transforms
        self.prepare = ConvertCocoPolysToMask(return_masks)
        elif backend == "local":
            self.backend = LocalBackend(root_dir)
        else:
            raise NotImplementedError

        self.catIds = catIds
        self.catIds = self.coco.getCatIds()
        '''
        Remapping to set background class to zero, so can support FasterRCNN models
        '''
        self.catIdtoCls = {
            catId: i+1 for i, catId in zip(range(len(self.catIds)), self.catIds)
        }
        self.clstoCatId = {
            v:k for k,v in self.catIdtoCls.items()
        }
        self.num_classes = len(list(self.catIdtoCls.values()))+1

    def __getitem__(self, idx):
        coco = self.coco
        img_id = self.ids[idx]
        ann_ids = coco.getAnnIds(imgIds=img_id, catIds=self.catIds)
        target = coco.loadAnns(ann_ids)
        path = coco.loadImgs(img_id)[0]["file_name"]
        img_bytes = BytesIO(self.backend.get(os.path.join(self.img_folder, path)))

        img = Image.open(img_bytes).convert("RGB")
        # img.save('test.png')
        image_id = self.ids[idx]
        target = {"image_id": image_id, "annotations": target}
        img, target = self.prepare(img, target)
        if self._transforms is not None:
            img, target = self._transforms(img, target)
        target["labels"] = torch.tensor(
                [self.catIdtoCls[l.item()] for l in target["labels"]], dtype=torch.int64
            )

        return img, target

    def __len__(self):
        return len(self.ids)
```

In [ ]:
def unwrap_collate_fn(batch):
    batch = list(zip(*batch))
    return tuple(batch)

data_dir = os.path.join('.', "data_downloaded")
print(data_dir)
dataset, num_classes = build_xview_dataset_filtered(image_set='train',args=AttrDict({
                                                'data_dir':data_dir,
                                                'backend':'local',
                                                'masks': None,
                                                }))
print("--num_classes: ",num_classes)
data_loader = torch.utils.data.DataLoader(
                                 dataset, 
                                 batch_size=16,
                                 batch_sampler=None,
                                 shuffle=True,
                                 num_workers=1, 
                                 collate_fn=unwrap_collate_fn)
print(data_dir)
dataset_test, _ = build_xview_dataset_filtered(image_set='val',args=AttrDict({
                                                'data_dir':data_dir,
                                                'backend':'local',
                                                'masks': None,
                                                }))

In [ ]:
cat_names

In [ ]:
num_classes

In [ ]:
# Labels used to translate category_id
# det_labels = ["Fixed-wing Aircraft", "Cargo Plane"]
det_labels = cat_names
print("det_labels: ",det_labels)
# Get one sample from the data set and show the image for validation
for sample in data_loader:
    img, targets = sample
    img = img[0].cpu()
    boxes = targets[0]['boxes'].cpu().numpy()
    labels = targets[0]['labels'].cpu().numpy()
    fig, ax = plt.subplots(1)
    ax.set_title("Example")
    ax.imshow(np.clip(img.permute(1, 2, 0), 0, 1))
    # Plot bounding boxes as rectangles on the image
    for box, label in zip(boxes, labels):
        x_min, y_min, x_max, y_max = box
        rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                             fill=False, edgecolor='red', linewidth=2)
        ax.add_patch(rect)
        ax.text(x_min, y_min - 5, f'Label: {det_labels[label-1]}', color='red', fontsize=10)

    plt.axis('off')
    plt.show()
    break

<h3>Step 4: Create DataLoader and model</h3>

In [ ]:
# Create pytorch data loader
data_loader = torch.utils.data.DataLoader(
                                 dataset, 
                                 batch_size=16,
                                 shuffle=True,
                                 num_workers=1, 
                                 collate_fn=unwrap_collate_fn)
print("NUMBER OF BATCHES IN COCO: ",len(data_loader))# 59143, 7392 for mini coco
data_loader_test = torch.utils.data.DataLoader(
                            dataset_test,
                            batch_size=4,
                            shuffle=False,
                            num_workers=1,
                            collate_fn=unwrap_collate_fn)

# Load FasterRCNN model (untrained) to GPU
model = build_frcnn_model(num_classes=num_classes).cuda()
device = 'cuda'

# Setup loss function and optimizer. Note: the loss function is built into the torchvision FasterRCNN model
optimizer = torch.optim.SGD(
            model.parameters(),
            lr=0.02,
            momentum=0.9,
            weight_decay=1e-4
        )

<h3>Run some training (overfitting on one batch)</h3>

In [ ]:
# Create iterator for data loader
dataiter = iter(data_loader)

# Get a single batch to overfit on
inputs, targets = dataiter.next()

In [ ]:
# Fit model on dataset 2 times
import time
loss_dict = {'loss_classifier':1, 'loss_box_reg': 1, 'loss_objectness': 1, 'loss_rpn_box_reg': 1, 'tr_time': 1}
# Initialize running sums and counts for each loss component
running_sums = {key: 0.0 for key in loss_dict.keys()}
running_counts = {key: 0 for key in loss_dict.keys()}
NUM_EPOCHS=40
for e in range(NUM_EPOCHS):
    for ind, batch in enumerate(data_loader):
        batch_time_start = time.time()
        images, targets = batch
        images = list(image.to(device, non_blocking=True) for image in images)
        targets = [{k: v.to(device, non_blocking=True) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses_reduced = sum(loss for loss in loss_dict.values())
        loss_value = losses_reduced.item()
        optimizer.zero_grad()
        losses_reduced.backward()
        optimizer.step()
        total_batch_time = time.time() - batch_time_start

        # Update running sums and counts for each loss component
        for key, value in loss_dict.items():
            running_sums[key] += value.item()
            running_counts[key] += 1

        # Calculate the running mean for each loss component
        running_means = {key: running_sums[key] / running_counts[key] for key in loss_dict.keys()}

        # Add the total batch time to the loss_dict
        running_means['tr_time'] = total_batch_time
        if ind%4 == 0:
            # Print the contents in a nice, formatted manner
            print(f"Epoch {e}/{NUM_EPOCHS}")
            print(f"{ind}/{len(data_loader)} ", end="")
            for key, value in running_means.items():
                if key not in ['accuracy', 'val_loss', 'val_accuracy']:
                    print(f"{key}: {value:.4f} - ", end="")
            print()
        # break

print('\nFinished Training')

In [ ]:
# torch.save(model.state_dict(),'model_{}_low_lr_3class.pth'.format(e))

<h3> Validate model trained in notebook</h3>

In [ ]:
import torch
import matplotlib.pyplot as plt

model.eval()
# det_labels = ["Fixed-wing Aircraft", "Cargo Plane"]
det_labels = cat_names
print("det_labels: ",det_labels)

with torch.no_grad():
    for sample in data_loader_test:
        images, targets = sample
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        print(len(images))
        fig, axs = plt.subplots(len(images), 2, figsize=(12, 6 * len(images)))

        for idx, (img, target) in enumerate(zip(images, targets)):
            img_np = img.clone().cpu().permute(1, 2, 0).numpy()

            # Ground truth
            axs[idx, 0].set_title("Ground Truth")
            axs[idx, 0].imshow(np.clip(img_np, 0, 1))
            boxes_gt = target['boxes'].cpu().numpy()
            labels_gt = target['labels'].cpu().numpy()
            # print(boxes_gt)
            for box, label in zip(boxes_gt, labels_gt):
                x_min, y_min, x_max, y_max = box
                rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                     fill=False, edgecolor='red', linewidth=2, alpha=0.5)
                axs[idx, 0].add_patch(rect)
                axs[idx, 0].text(x_min, y_min - 5, f'Label: {det_labels[label-1]}', color='red', fontsize=10)

            # Model prediction
            axs[idx, 1].set_title("Model Prediction")
            axs[idx, 1].imshow(np.clip(img_np, 0, 1))
            outputs = model([img])
            print(outputs[0])
            if outputs[0]['boxes'].nelement() !=0:
                outputs = [{k: v[0].to('cpu') for k, v in output.items()} for output in outputs]
                boxes_pred = [outputs[0]['boxes'].numpy()] if not isinstance(outputs[0]['boxes'].numpy(),list) else outputs[0]['boxes'].numpy()
                labels_pred = [outputs[0]['labels'].numpy()] if not isinstance(outputs[0]['labels'].numpy(),list) else outputs[0]['labels'].numpy()
                scores_pred = [outputs[0]['scores'].numpy()] if not isinstance(outputs[0]['scores'].numpy(),list) else outputs[0]['scores'].numpy()

                # Plot bounding boxes with scores greater than 0.05 as rectangles on the image
                # print(boxes_pred, labels_pred, scores_pred)
                for box, label, score in zip(boxes_pred, labels_pred, scores_pred):
                    if score > 0.005:
                        x_min, y_min, x_max, y_max = box
                        rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                             fill=False, edgecolor='green', linewidth=2, alpha=0.5)
                        axs[idx, 1].add_patch(rect)
                        axs[idx, 1].text(x_min, y_min - 5, f'Label: {det_labels[label-1]}, Score: {score:.2f}', color='green', fontsize=12)

                plt.axis('off')
                # plt.show()
            else:
                pass
                # plt.axis('off')
        plt.show()
            # break


# How ML Engineers integrate models into MLDE
<img src="./img/determined_workflow.png" alt="Enterprise Machine Learning platform architecture" width="850">


```python
from determined.pytorch import (
    DataLoader,
    LRScheduler,
    PyTorchTrial,
    PyTorchTrialContext,
    MetricReducer,
)
...
class ObjectDetectionTrial(PyTorchTrial):
    def __init__(self, context: PyTorchTrialContext) -> None:
        self.context = context
        self.hparams = AttrDict(self.context.get_hparams())
        print(self.hparams) 
        ...
        self.model = self.context.wrap_model(model)
        
        # wrap optimizer
        self.optimizer = self.context.wrap_optimizer(optimizer)

        scheduler_cls = WarmupWrapper(MultiStepLR)
        ...
        self.scheduler = self.context.wrap_lr_scheduler(
            scheduler, step_mode=LRScheduler.StepMode.MANUAL_STEP
        )
        ...
    def build_training_data_loader(self) -> DataLoader:
        ...
        return data_loader

    def build_validation_data_loader(self) -> DataLoader:
        ...
        
        return data_loader_test
    
    def train_batch(self, batch: TorchData, epoch_idx: int, batch_idx: int) -> Dict[str, torch.Tensor]:
        batch_time_start = time.time()
        images, targets = batch
        ...
        loss_dict = self.model(images, targets)
        losses_reduced = sum(loss for loss in loss_dict.values())
        loss_value = losses_reduced.item()
        self.context.backward(losses_reduced)
        self.context.step_optimizer(self.optimizer)
        self.scheduler.step()
        loss_dict['lr'] = self.scheduler.get_lr()[0]

        return loss_dict
    
    def evaluate_batch(self, batch: TorchData,batch_idx: int) -> Dict[str, Any]:
        images, targets = batch
        model_time_start = time.time()
        # loss_dict, outputs = self.model(images, targets)
        loss_dict = {}
        loss_dict['eval_loss']=0.0
        outputs = self.model(images, targets)
        ...
        self.reducer.update(result)
        ...
        return loss_dict
```

<h1>Migrate Training into MLDE</h1>

<img src="./img/platform_step2.png" alt="Enterprise Machine Learning platform architecture" width="850">

<h3>Connect to our cluster and submit experiment on our AI at Scale Training platform</h3>
This will take a few minutes because we are completing the following tasks:

* Uploading experiment artifacts
* Cluster allocates GPU resources and downloads docker container that was specified in experiment config 
* some additional python packages need to be installed
* then training kicks off

In [ ]:
# Create experiment using yaml file and submit to MLDE
exp = det.create_experiment(config="./experiment/const.yaml", model_dir="./experiment/")
print(f"started experiment {exp.id}")

# Wait for experiment to complete and print exit status
exit_status = exp.wait()
print(f"experiment completed with status {exit_status}")

# Get the best Checkpoint of the experiment and print uuid
best_checkpoint = exp.top_checkpoint()
best_checkpoint_uuid = best_checkpoint.uuid
print(f"Best checkpoint was {best_checkpoint_uuid}")

<h3>Validate model trained on MLDE</h3>

In [ ]:
# Pull model checkpoint from Determined.AI using UUID
best_checkpoint = det.get_checkpoint(best_checkpoint_uuid)
print("Best checkpoint retrieved.")

In [ ]:
# Load checkpoint to model
path = best_checkpoint.download()
print(path)
N_CLASSES = num_classes + 1# Add one to include background class
trained_model = build_frcnn_model(N_CLASSES)
trained_model = load_model(trained_model,path)
print("Checkpoint loaded into model.")

In [ ]:
import torch
import matplotlib.pyplot as plt
trained_model.to('cuda')
trained_model.eval()
# det_labels = ["Fixed-wing Aircraft", "Cargo Plane"]

det_labels = cat_names
print("det_labels: ",det_labels)

device = 'cuda'
with torch.no_grad():
    for sample in data_loader_test:
        images, targets = sample
        images = [img.to(device) for img in images]
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        fig, axs = plt.subplots(len(images), 2, figsize=(12, 6 * len(images)))

        for idx, (img, target) in enumerate(zip(images, targets)):
            img_np = img.clone().cpu().permute(1, 2, 0).numpy()

            # Ground truth
            axs[idx, 0].set_title("Ground Truth")
            axs[idx, 0].imshow(np.clip(img_np, 0, 1))
            boxes_gt = target['boxes'].cpu().numpy()
            labels_gt = target['labels'].cpu().numpy()
            # print(boxes_gt)
            for box, label in zip(boxes_gt, labels_gt):
                x_min, y_min, x_max, y_max = box
                rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                     fill=False, edgecolor='red', linewidth=2, alpha=0.5)
                axs[idx, 0].add_patch(rect)
                axs[idx, 0].text(x_min, y_min - 5, f'Label: {det_labels[label-1]}', color='red', fontsize=10)

            # Model prediction
            axs[idx, 1].set_title("Model Prediction")
            axs[idx, 1].imshow(np.clip(img_np, 0, 1))
            outputs = trained_model([img])
            outputs = [{k: v[0].to('cpu') for k, v in output.items()} for output in outputs]
            boxes_pred = [outputs[0]['boxes'].numpy()] if not isinstance(outputs[0]['boxes'].numpy(),list) else outputs[0]['boxes'].numpy()
            labels_pred = [outputs[0]['labels'].numpy()] if not isinstance(outputs[0]['labels'].numpy(),list) else outputs[0]['labels'].numpy()
            scores_pred = [outputs[0]['scores'].numpy()] if not isinstance(outputs[0]['scores'].numpy(),list) else outputs[0]['scores'].numpy()

            # Plot bounding boxes with scores greater than 0.05 as rectangles on the image
            # print(boxes_pred, labels_pred, scores_pred)
            for box, label, score in zip(boxes_pred, labels_pred, scores_pred):
                if score > 0.05:
                    x_min, y_min, x_max, y_max = box
                    rect = plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                         fill=False, edgecolor='green', linewidth=2, alpha=0.5)
                    axs[idx, 1].add_patch(rect)
                    axs[idx, 1].text(x_min, y_min - 5, f'Label: {det_labels[label-1]}, Score: {score:.2f}', color='green', fontsize=12)

        plt.axis('off')
        plt.show()
        break


<h1>Migrate Training and Inference into Scalable, Automated E2E Ops: Model deployment for inference</h1>

<img src="./img/platform_step3.png" alt="Enterprise Machine Learning platform architecture" width="850">

In [ ]:
client = python_pachyderm.Client(
    host=pachyderm_host, port=pachyderm_port, auth_token=token
)

In [ ]:
# Create Pipeline:
import json
PATH = 'pdk-use-cases-dev/object-detection/pipelines/training-pipeline.json'
with open(PATH,'r') as file:
    spec_d = json.loads(file.read())
    spec = python_pachyderm.parse_dict_pipeline_spec(spec_d)
spec

In [ ]:
client.create_pipeline(
    pipeline_name=spec.pipeline.name,
    transform=spec.transform,
    project_name=project,
    input=spec.input,
    description=spec.description,
    pod_patch = spec.pod_patch
)

In [ ]:
import json
PATH = 'pdk-use-cases-dev/object-detection/pipelines/deployment-pipeline.json'
with open(PATH,'r') as file:
    spec_d = json.loads(file.read())
    spec = python_pachyderm.parse_dict_pipeline_spec(spec_d)
spec

In [ ]:
client.create_pipeline(
    pipeline_name=spec.pipeline.name,
    transform=spec.transform,
    project_name=project,
    input=spec.input,
    description=spec.description,
    pod_patch = spec.pod_patch
)

Show how on the MLDM pipeline UI, both training and deployment pipeline are created. We will need to wait a few minutes for each pipeline step to complete, 

## Lets Look at model predicitions with our deployed model in another notebook!

# Trigger End to End Pipeline Execution with updated dataset

In [ ]:
import python_pachyderm
from python_pachyderm.service import pps_proto

In [ ]:
# Define variables for Pachyderm connection
pachyderm_host = "10.182.1.45"
pachyderm_port = 80
repo = "data"
branch = "master"
project = "object-detection-demo"
download_dir = "./xview_dataset_full"
data_dir = './data/xview_2class_full/'
token = " "

## Here we have a new version of the dataset that includes more labels (88 annotations)

In [ ]:
FILE_PATH=f'{data_dir}/train_images_rgb_no_neg_filt_32/train_640_02_filtered_32.json'
(num_annotations, 
 cat_ids, 
 cat_names, 
 annotations_per_category, 
 min_annotations, 
 max_annotations, 
 average_annotations, 
 min_annotation_area,
 max_annotation_area,
 avg_annotation_area,
 min_annotation_area_per_category, 
 max_annotation_area_per_category) = calculate_coco_stats(FILE_PATH)
print("Number of images:", num_annotations)
print(f"Number of Unique Categories: {len(cat_ids)}")
print(cat_ids)  # The IDs are not necessarily consecutive.

print("Category IDs:")
print("Categories Names: ", cat_names)
# Print or use annotations_per_category as needed
print("Number of annotations per category:")
print(annotations_per_category)
# Print or use the calculated metrics as needed
print("Minimum number of annotations for an image:", min_annotations)
print("Maximum number of annotations in an image:", max_annotations)
print("Average number of annotations per image:", average_annotations)

print("Minimum annotation area:", min_annotation_area)
print("Maximum annotation area:", max_annotation_area)
print("Average annotation area:", avg_annotation_area)

print("\nMinimum annotation area per category:")
print(min_annotation_area_per_category)

print("\nMaximum annotation area per category:")
print(max_annotation_area_per_category)

We will show how when we upload an updated version of our dataset, the entire end-to-end training and deployment pipeline kicks off

In [ ]:
client = python_pachyderm.Client(
    host=pachyderm_host, port=pachyderm_port, auth_token=token
)
source_dir  = data_dir
def insert_data(client,name,source_data_dir,project):
    print("Inserting {} data...".format(name))
    with client.commit(name, "master",project_name=project) as c:
        # data_dir = "{}_data".format(name)
        python_pachyderm.put_files(client, source_data_dir, c, "/")
                
        return c
# name = "objdet-data2"
d_commit = insert_data(client,repo,source_data_dir=download_dir
                       
                       ,project=project)

# Wait for the commits to finish
print("Waiting for commits to finish...")
for commit in [client.wait_commit(c.id)[0] for c in [d_commit]]:
    print(commit)

## Run cell to clean up environment

In [ ]:
# delete pachyderm pipelines
import json

PATH = 'pdk-use-cases-dev/object-detection/pipelines/deployment-pipeline.json'
with open(PATH,'r') as file:
    spec_d = json.loads(file.read())
    spec = python_pachyderm.parse_dict_pipeline_spec(spec_d)
# delete deployment pipeline
client.delete_pipeline(
        pipeline_name=spec.pipeline.name,
        force = False,
        keep_repo = False,
        project_name = 'object-detection-demo',
    )
print("Deleted pipeline: {}".format(spec.pipeline.name))


PATH = 'pdk-use-cases-dev/object-detection/pipelines/training-pipeline.json'
with open(PATH,'r') as file:
    spec_d = json.loads(file.read())
    spec = python_pachyderm.parse_dict_pipeline_spec(spec_d)

# delete training pipeline
client.delete_pipeline(
        pipeline_name=spec.pipeline.name,
        force = False,
        keep_repo = False,
        project_name = 'object-detection-demo',
    )
print("Deleted pipeline: {}".format(spec.pipeline.name))

# delete pachyderm repo
client.delete_repo(
        repo_name=repo, 
        force = False, 
        project_name = 'object-detection-demo',
    )
print("Deleted repo: {}".format(repo))